# Аналитические и численные методы линейной регрессии

В этом ноутбуке рассматриваются различные подходы к решению задачи линейной регрессии с помощью **аналитических (непараметрических) методов** и параметрического метода LinearRegression из scikit-learn

In [4]:
import numpy as np
import pandas as pd

# Датасет: 10 наблюдений, один признак
data_single = {
    'x': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'y': [3.5, 6.8, 9.2, 12.0, 15.1, 18.3, 21.0, 24.2, 27.1, 30.5]
}

df_single = pd.DataFrame(data_single)
print("Датасет (один признак):")
print(df_single)

Датасет (один признак):
    x     y
0   1   3.5
1   2   6.8
2   3   9.2
3   4  12.0
4   5  15.1
5   6  18.3
6   7  21.0
7   8  24.2
8   9  27.1
9  10  30.5


## 1. Аналитическое решение парной линейной регрессии

Для случая **ОДНОГО** признака $ x $ и целевой переменной $ y $ можно найти оптимальные параметры модели $ y = mx + b $ **аналитически**, без итераций.

###  Цель
Найти такие $ m $ (наклон) и $ b $ (свободный член), которые минимизируют сумму квадратов ошибок (MSE).

---

###  Формулы МНК (метод наименьших квадратов)

Для $ n $ наблюдений:

- **Наклон $ m $:**
  $$
  m = \frac{n \sum (x_i y_i) - \sum x_i \sum y_i}{n \sum (x_i^2) - (\sum x_i)^2}
  $$

- **Свободный член $ b $:**
  $$
  b = \bar{y} - m \bar{x}
  $$
  где $ \bar{x} $ и $ \bar{y} $ — средние значения.

Эта формула — частный случай метода наименьших квадратов (МНК), выведенный только для случая одного признака x .
Она не работает при двух и более признаках, потому что: 

- теряется возможность учитывать взаимное влияние признаков,
- нельзя выразить несколько коэффициентов в одной скалярной формуле.


###  Реализация формулы в коде:

In [8]:
points = list(df_single.itertuples(index=False))

n = len(points)

# Вычисляем m (наклон)
m = (n * sum(p.x * p.y for p in points) - sum(p.x for p in points) * sum(p.y for p in points)) / \
    (n * sum(p.x**2 for p in points) - (sum(p.x for p in points))**2)

# Вычисляем b (свободный член)
b = (sum(p.y for p in points) / n) - m * (sum(p.x for p in points) / n)

print(f"\nМетод 1 (ручная формула):")
print(f"y = {m:.3f}x + {b:.3f}")


Метод 1 (ручная формула):
y = 2.972x + 0.427


##  2. Аналитическое (матричное) решение парной и множественной линейной регрессии (МНК)

Этот метод позволяет найти оптимальные параметры модели \( y = mx + b \) аналитически, используя линейную алгебру. Он работает не только для одного признака, но и для любого числа признаков — это универсальный подход, основанный на методе наименьших квадратов (МНК).

### Математическая формула

Оптимальные параметры $ \mathbf{w} $ находятся по формуле метода наименьших квадратов:

$$
\mathbf{w} = (X^T X)^{-1} X^T \mathbf{y}
$$

Где:
- $ X $ — матрица признаков с добавленным столбцом единиц (для свободного члена $ b $)
- $ \mathbf{y} $ — вектор целевых значений
- $ \mathbf{w} = [b, m] $ — вектор параметров модели


Хотя в теории решение выражается формулой, на практике в библиотеках, включая `scikit-learn`, она **не используется напрямую** из-за численной неустойчивости и может приводить к значительным ошибкам в присутствии мультиколлинеарности, шума или при большом количестве признаков.

### Реализация формулы в коде:

In [15]:
X = np.column_stack([np.ones(n), df_single['x']])  # [1, x]
y = df_single['y'].values

# Аналитическое решение: w = (X^T X)^{-1} X^T y
w = np.linalg.inv(X.T @ X) @ X.T @ y

b_mnk, m_mnk = w[0], w[1]

print(f"\nМетод 2 (матричная формула):")
print(f"y = {m_mnk:.3f}x + {b_mnk:.3f}")


Метод 2 (матричная формула):
y = 2.972x + 0.427


## 3. Метод  LinearRegression из  scikit-learn

Внутри вызывается функция `numpy.linalg.lstsq` или аналог из `scipy`, которая решает задачу наименьших квадратов по умолчанию с помощью SVD.

Матрица $ X $ представляется в виде:
$$
X = U \Sigma V^T
$$
где:
- $ U $ и $ V $ — ортогональные матрицы,
- $ \Sigma $ — диагональная матрица сингулярных значений.

Решение для вектора весов $ \mathbf{w} $ находится как:
$$
\mathbf{w} = V \Sigma^{-1} U^T \mathbf{y}
$$

### Почему не используется формула с обратной матрицей?

Несмотря на математическую корректность формулы $ (X^T X)^{-1} X^T y $, её применение в реальных условиях затруднено по следующим причинам:

1. **Численная неустойчивость**  
   При близкой к линейной зависимости признаков (мультиколлинеарность) матрица $ X^T X $ становится плохо обусловленной, и её обращение приводит к большим ошибкам.

2. **Потеря точности при умножении**  
   Вычисление $ X^T X $ удваивает ошибку округления, что особенно критично при большом масштабе данных.

3. **Ограничения по размерности**  
   Если число признаков превышает число наблюдений, матрица $ X^T X $ вырождена и необратима.

4. **Риск переполнения**  
   При больших значениях элементов $ X $ произведение $ X^T X $ может привести к переполнению.


### Преимущества использования SVD

- Высокая численная устойчивость.
- Возможность работы с вырожденными и плохо обусловленными матрицами.
- Поддержка случаев, когда число признаков больше числа объектов.
- Возможность контроля устойчивости через параметр `rcond` (порог усечения малых сингулярных значений).

In [18]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Данные: один признак
X_single = df_single[['x']] 
y_single = df_single['y']

# Модель
model = LinearRegression()
model.fit(X_single, y_single)

# Результат
m_sk = model.coef_[0]    # наклон
b_sk = model.intercept_  # свободный член

print(f"\nМетод 3 (LinearRegression):")
print(f"y = {m_sk:.3f}x + {b_sk:.3f}")


Метод 3 (LinearRegression):
y = 2.972x + 0.427


## **Пример для ДВУХ признаков через матричную формулу:**

In [10]:
data_multi = {
    'x1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'x2': [2, 3, 3, 4, 4, 5, 5, 5, 5, 5],
    'y':  [4.0, 7.0, 9.5, 12.5, 15.0, 18.0, 21.0, 24.0, 26.5, 29.0]
}

df_multi = pd.DataFrame(data_multi)
print("\nДатасет (два признака):")
print(df_multi)


Датасет (два признака):
   x1  x2     y
0   1   2   4.0
1   2   3   7.0
2   3   3   9.5
3   4   4  12.5
4   5   4  15.0
5   6   5  18.0
6   7   5  21.0
7   8   5  24.0
8   9   5  26.5
9  10   5  29.0


In [20]:
n = len(df_multi)

# Матрица X: [1, x1, x2]
X_multi = np.column_stack([np.ones(n), df_multi['x1'], df_multi['x2']])
y_multi = df_multi['y'].values

# Решение: w = (X^T X)^{-1} X^T y
w_multi = np.linalg.inv(X_multi.T @ X_multi) @ X_multi.T @ y_multi

b_multi, w1, w2 = w_multi[0], w_multi[1], w_multi[2]

print(f"\nМножественная регрессия (матричная формула):")
print(f"y = {w1:.3f}·x1 + {w2:.3f}·x2 + {b_multi:.3f}")


Множественная регрессия (матричная формула):
y = 2.752·x1 + 0.154·x2 + 0.885


## **Пример для ДВУХ признаков из "коробки":**

In [19]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Данные
X_multi = df_multi[['x1', 'x2']]  # признаки
y_multi = df_multi['y']           # целевая переменная

# Модель "из коробки"
model = LinearRegression()
model.fit(X_multi, y_multi)

# Получаем коэффициенты
w1_sk = model.coef_[0]   # коэффициент при x1
w2_sk = model.coef_[1]   # коэффициент при x2
b_sk = model.intercept_  # свободный член

print(f"\nМножественная регрессия (LinearRegression):")
print(f"y = {w1_sk:.3f}·x1 + {w2_sk:.3f}·x2 + {b_sk:.3f}")


Множественная регрессия (LinearRegression):
y = 2.752·x1 + 0.154·x2 + 0.885
